### Install the modified version of Geopandas

In [1]:
!pip3 install git+https://github.com/marouaneloutfi/geopandas.git@polyline-M-support
!pip3 install git+https://github.com/marouaneloutfi/Fiona.git@polyline-M-support

  Cloning https://github.com/marouaneloutfi/geopandas.git (to polyline-M-support) to /tmp/pip-enlk2de4-build
  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/38/b6/b53f19062afd49bb5abd049aeed36f13bf8d57ef8f3fa07a5203531a0252/Shapely-1.6.4.post2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/50/f7/9899f8a9a2e38601472fe1079ce5088f58833221c8b8507d8b5eafd5404a/Fiona-1.8.13-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5f/34/69acc7e9c3d1a4b89bca174af9242f86c608bef6e181f4d7d172d0f940a4/pyproj-2.4.2.post1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonho

### Import The necessary modules

In [3]:
import geopandas

### Read the shape file containing the polyline M geometries into a geo dataframe 

In [6]:
gdf = geopandas.read_file('../data/polylineM/asv_lines_Calibrated.shp')

_**The M values are stored seperatly for each feature (Geometry) in its 'properties' dict under the key measurements**_

In [7]:
print(gdf.head())

         lmarkname  year  jday discover        divisors       orig_fname  \
0  asv06_154ap_000  2006   154       no  -600000/600000  apallchicola000   
1  asv06_154ap_001  2006   154       no  -600000/600000  apallchicola001   
2  asv06_154ap_002  2006   154       no  -600000/600000  apallchicola002   
3  asv06_154ap_003  2006   154       no  -600000/600000  apallchicola003   
4  asv06_154ap_004  2006   154       no  -600000/600000  apallchicola004   

                                            geometry  \
0  LINESTRING (-84.84812700000001 29.746692, -84....   
1  LINESTRING (-84.847802 29.7465, -84.847785 29....   
2  LINESTRING (-84.84593 29.745267, -84.84593 29....   
3  LINESTRING (-84.847753 29.743627, -84.847737 2...   
4  LINESTRING (-84.842692 29.745258, -84.842708 2...   

                                        measurements  
0  [1.0, 3.0, 18.0, 46.0, 74.0, 103.0, 118.0, 132...  
1  [1.0, 14.0, 27.0, 42.0, 70.0, 84.0, 98.0, 128....  
2  [1.0, 15.0, 29.0, 43.0, 57.0, 73.0, 87

**_To easily combine the measurements with their corresponding coordinates in the form of [x,y,M],
The following functions are provided_**

In [8]:
# returns a geometry dict with measurements value embeded in the coordinates [x, y, M]
def zip_geometry(geom, m_values):
    geom['coordinates'] = [ coord + (m,) for coord, m in zip(geom['coordinates'], m_values)]
    return geom


# returns a geometry dict and the measurement values SEPERATLY
def unpack_geometry(geom):
    m_array, geom['coordinates'] = [], []
    for coord in geom['coordinates']:
        coords.append(coord[:2])
        m_array.append(coord[2])
    geom['type'] = geom['type'].rstrip('M')
    return geom, m_array


_**Here we use the zip_geometry function to couple each coordinate with its measurement and generate a Geojson file, in this case the Measurement values are taking the place of Z-axis values**_

In [18]:
geojson_compact = {}
geojson_compact['features'] = [ {'type': 'Feature', 'geometry': zip_geometry(f['geometry'], f['properties']['measurements'])}
                for f in gdf.iterfeatures() ]

json.dumps(geojson_compact['features'][0])

'{"type": "Feature", "geometry": {"type": "LineString", "coordinates": [[-84.848127, 29.746692, 1.0], [-84.848127, 29.746683, 3.0], [-84.848112, 29.746678, 18.0], [-84.848095, 29.74667, 46.0], [-84.848078, 29.746658, 74.0], [-84.848062, 29.746647, 103.0], [-84.848062, 29.746642, 118.0], [-84.848047, 29.746642, 132.0], [-84.84803, 29.746633, 161.0], [-84.848013, 29.746627, 189.0], [-84.847997, 29.746622, 203.0], [-84.847982, 29.746613, 231.0], [-84.847965, 29.746605, 260.0], [-84.847948, 29.746602, 289.0], [-84.847932, 29.746593, 318.0], [-84.847917, 29.746585, 345.0], [-84.8479, 29.746577, 374.0], [-84.847883, 29.746568, 402.0], [-84.847867, 29.746557, 431.0], [-84.84785, 29.746545, 459.0], [-84.84785, 29.74654, 474.0]]}}'